In [4]:
# plot the audio data
# use popup window
%matplotlib qt
import matplotlib.pyplot as plt

from utils import *

In [ ]:
rate = 48000
seconds = 10 / 400
freq_n = 12
symbol_freq = 1000
symbol_time = 1 / symbol_freq
symbol_samples = int(symbol_time * rate)
cp_samples = int(0.2 * symbol_samples)
pause_samples = 0

t = np.linspace(0, symbol_time, symbol_samples, endpoint=False)
sinew = sine_wave(freq_n * symbol_freq)(t)
cosinew = sine_wave(freq_n * symbol_freq, phase=np.pi/2)(t)

one = np.concatenate((sinew[-cp_samples:], sinew, np.zeros(pause_samples)))
zero = np.concatenate((cosinew[-cp_samples:], cosinew, np.zeros(pause_samples)))

plt.plot(one)
plt.plot(zero)

The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x62b07078b160


In [8]:
for i in range(10):
    bin = np.random.randint(0, 2, 100)
    # print(bin)
    y = np.concatenate([one if b == 1 else zero for b in bin])

    record_wave, rate, raw_record_wave = play_and_record_precise2(y, rate)
    # record_wave = record_wave / np.max(np.abs(record_wave))

    # plt.plot(raw_record_wave)
    # plt.plot(64325 + np.array([i for i in range(len(record_wave))]), 1 + record_wave)
    # plt.plot(64325 + np.array([i for i in range(len(y))]), -1 + 0.5 * y)
    # plt.plot(record_wave)
    # plt.plot(y)

    # plt.plot(sinew)

    recv_bin = []
    for i in range(len(bin)):
        start = i * (cp_samples + symbol_samples)
        real_start = start + cp_samples
        real_end = real_start + symbol_samples

        wave = record_wave[real_start:real_end]
        fft_result = np.fft.rfft(wave)

        real_mag = np.abs(np.real(fft_result[freq_n]))
        imag_mag = np.abs(np.imag(fft_result[freq_n]))
        if real_mag > imag_mag:
            recv_bin.append(0)
        else:
            recv_bin.append(1)

        # plt.axvline(real_start, color='r')
        # plt.axvline(real_end, color='g')

    recv_bin = np.array(recv_bin)

    # print(bin)
    # print(recv_bin)
    similarity = np.sum(bin == recv_bin) / len(bin)
    print(similarity)

1.0


KeyboardInterrupt: 